In [52]:
# S3 prefix
BUCKET = '{BUCKET}'
PREFIX = f"s3://{BUCKET}/mlproject/training-dataset/heart_diseases_normalizedml_withtarget_dataset.csv"

import sagemaker
import os
from sagemaker import get_execution_role

sagemaker_session = sagemaker.Session(default_bucket=BUCKET )

# Get a SageMaker-compatible role used by this Notebook Instance.
role = get_execution_role()


In [78]:
from sagemaker.sklearn.estimator import SKLearn

script_path = 'deployment_GMM.py'

sklearn = SKLearn(
    entry_point=script_path,
    train_instance_type="ml.m5.large",
    framework_version='0.23-1',
    role=role,
    sagemaker_session=sagemaker_session)

sklearn.fit({'train' : PREFIX})


In [76]:
import numpy
from sagemaker.sklearn.estimator import SKLearn,SKLearnModel


endpoint_name = 'mgmm-scikit-endpoint'
predictor = sklearn.deploy(instance_type="ml.c4.xlarge", initial_instance_count=1,endpoint_name=endpoint_name)

Parameter image will be renamed to image_uri in SageMaker Python SDK v2.


-----------------!

In [77]:
import boto3 
import json

client = boto3.client('sagemaker-runtime')


response = client.invoke_endpoint(
    EndpointName=endpoint_name,  
#     CustomAttributes=custom_attributes, 
    ContentType='text/csv',
#     Accept=accept,
    Body= '30,0,7861,0,38,0,263358,1,136,1,0,6'
    )

result = response['Body']
model_prediction = json.loads(result.read())
print(model_prediction)
print(model_prediction['instances'][0]['features'])


{'instances': [{'features': [0.0, 1.0]}]}
[0.0, 1.0]
